In this lesson, we will demonstrate how to solve 2D Stokes equations with th PT method 

### Philosophy
The philosophy of our PT method is to write numerical code corresponding to the equations'  mathematical formulation. As long as you know the equations,
you should know how to write the code to solve it! 



Here is the Stokes Equations that usually solved in geodynamics: viscous rheology; incompressible; boussineqq approximation;

   $$ \nabla _j (\tau_{ij} -P\delta_{ij})  -\rho g= 0   \tag{1} $$
   $$\nabla _k V_k = 0  \tag{2} $$
   whereas;
   $$\tau_{ij}=2 \eta_s \cdot (\frac{1}{2}(\nabla _i V_j+\nabla _j V_i)-\frac {1}{3}\delta_{ij}\nabla _k V_k)  \tag {3} $$


        


 The formulaiton for 2D case: 

   $$ \frac {\partial (\tau_ {xx} -P)} {\partial x} +\frac {\partial \tau_{xy}}{\partial y} -\rho g_x= 0  \tag{4} $$
   $$ \frac {\partial (\tau_ {yy} -P)} {\partial y}+\frac {\partial \tau_ {xy}} {\partial x} -\rho g_y= 0  \tag{5} $$
   $$ \nabla _k V_k=\frac {\partial V_ {x}} {\partial x}+\frac {\partial V_ {y}} {\partial y}  = 0  \tag{6} $$

   whereas;
   $$\tau_{xy}=2 \eta_s \cdot (\frac{1}{2}(\frac {\partial V _x}{\partial y} +\frac {\partial V _y}{\partial x} ) $$
   $$\tau_{xx}=2 \eta_s \cdot (\frac {\partial V _x}{\partial x}- \frac{1}{3}\nabla _k V_k)  $$
   $$\tau_{yy}=2 \eta_s \cdot (\frac {\partial V _y}{\partial y}- \frac{1}{3}\nabla _k V_k)   $$

Consider the incompressible assumption, $\nabla _k V_k=0$, the deritoric stress: 
   $$\tau_{xy}=2 \eta_s \cdot (\frac{1}{2}(\frac {\partial V _x}{\partial y} +\frac {\partial V _y}{\partial x} ) \tag {7} $$
   $$\tau_{xx}=2 \eta_s \cdot (\frac {\partial V _x}{\partial x}) \tag {8} $$
   $$\tau_{yy}=2 \eta_s \cdot (\frac {\partial V _y}{\partial y}) \tag {9} $$

The PT formulation to solve $V_x$ at a specific physical time through pseudo time stepping is: 

  $$ \frac{\partial V_x}{\partial \tau}=\frac {\partial (\tau_ {xx} -P)} {\partial x} +\frac {\partial \tau_{xy}}{\partial y} -\rho g_x  \tag{10}$$
  $$ \frac{\partial V_y}{\partial \tau}= \frac {\partial (\tau_ {yy} -P)} {\partial y}+\frac {\partial \tau_ {xy}} {\partial x} -\rho g_y  \tag{11} $$
  $$ \frac{\partial P}{\partial \tau_p} = -(\frac {\partial V_ {x}} {\partial x}+\frac {\partial V_ {y}} {\partial y}) \tag{12} $$


Correspondingly, the octave code is:
<pre>        div = (diff(Vx,1,1)/dx+diff(Vy,1,2)/dy);
<pre>        Exx = diff(Vx,1,1)/dx-1/3*div;
<pre>        Eyy = diff(Vy,1,2)/dy-1/3*div;
<pre>        Exy = (diff(Vx(2:nx,:),1,2)/dy+diff(Vy(:,2:ny),1,1)/dx)*0.5;
<pre>        Txx = 2.0*mu*(Exx+beta_n*div);   %(nx,ny)
<pre>        Tyy = 2.0*mu*(Eyy+beta_n*div);   %(nx,ny)
<pre>        Txy(2:nx,2:ny) =2.0*mu*Exy;
<pre>        RVx1=(diff(Txx(:,2:ny-1),1,1)/dx+diff(Txy(2:nx,2:ny),1,2)/dy- diff(P(:,2:ny-1),1,1)/dx);
<pre>        RVx2=RVx1+dampx*RVx2;
<pre>        RVy1=(diff(Tyy(2:nx-1,:),1,2)/dy+diff(Txy(2:nx,2:ny),1,1)/dx- diff(P(2:nx-1,:),1,2)/dy- rhosg);
<pre>        RVy2=RVy1+dampx*RVy2;
<pre>        Vx(2:nx,2:ny-1) = Vx(2:nx,2:ny-1)+dtau*RVx2;
<pre>        Vy(2:nx-1,2:ny) = Vy(2:nx-1,2:ny)+dtau*RVy2;
<pre>        P = P - dtauP*div;  %P = P - mean(P(:));

Based on the understanding of PT methods from previous lesson, I hope you can see that the above code matches well with the formulation:Eq.4)-Eq.12).
However, you would noticed there are two new things.

### 1. New parameter $\beta_n$, which is the artifical bulk viscosity. 
Test different value to see how it affect the convergence! 
Would it change the solution that as we modified the formulation? No, it won't change the solution as $\nabla V$ is approaching 0 when the code converges.
      


### 2. A different pseudo timestep for P ($d\tau_p$) is use than the pseudo timestep for Vx and Vy ($d\tau$) 
Therefore, we need to define both timestep in the code.

The code to define the pseudo timestep and dampening parameters are: 
<pre>            D            = 2*mu*(2/3+beta_n)+mu; # equvilent diffusion coefficient.
<pre>            dtau         = CFL*dx*dx/D/2.1/1; # dt from damped wave equation. % it works with D/2.1, very fast! But be ready to change to D/4.1 if not converged!
<pre>            a            = dtau;
<pre>            bx           = 1* pi*omega/nx;
<pre>            by           = 1* pi*omega/ny;  % include beta_n for pressure equation!
<pre>            dampx        = 1 - bx;
<pre>            dampy        = 1 - by;
<pre>            dtauP        = 0.95*2*mu*ndim*(1+1*beta_n)/nx;  

The supplemental material DeriveDampening2DStokes2nd.ipynb provide a derivation for the dampening parameters (dampx,dampy) and $d\tau$. You may find it useful. 
However, it doesn't provide an explanation for the choice of $d\tau_p$. You can take it as a empirical parameter.  Try to find the best value for your problem!



### Practice：  

<pre>           1. test differnt dampening parameters and see how does they affect the convergence
<pre>           2. use different intial consition.
<pre>           3. implement different boundary conditions.
<pre>           4. You should try to write the own 2D code from scratch. 


### Material for this lesson:

<pre>    Stokes2D_2nddamp.m 
<pre>    DeriveDampening2DStokes2nd.ipynb